<a href="https://colab.research.google.com/github/PUBPOL-2130/notebooks/blob/week5-fix/helper-notebooks/Week5-supplemental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q geopandas networkx

In [ ]:
%config InlineBackend.figure_formats = ["svg"]
import base64
import io
import json
import requests

import pandas as pd; pd.set_option('display.max_rows', 500)
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx

from shapely import Point

# Week 5: Supplemental Materials

In this notebook, we will first load the SIPRI data and then cut it down considerably to avoid timeout errors when creating our own Google sheet. This may help you with creating your own Google sheet when creating a flow map with Flowmapblue.

## Data cleaning

We'll start with preparing our data exactly as in the Week 5 lab.

**Note: We have received reports of errors in the next cell. Some of these errors are due to the fact that the SIPRI data may have gone offline. If you are encountering errors, you can use a backup version of the data.**

Please set the following variable `download_raw_data` to `False` if you want to use a backup version of the data.

In [ ]:
# set the following variable to false if using backup data
download_raw_data=False

In [ ]:
# loading raw data
if download_raw_data:
    raw_data = requests.post(
        "https://github.com/PUBPOL-2130/notebooks/blob/main/data/sipri_arms_transfers.json",
        json={"filters": []},
    ).json()

In [ ]:
# converting from base6
if download_raw_data:
    csv_lines = base64.b64decode(raw_data["bytes"]).decode("iso-8859-1").split("\n")
    csv_lines[:15]

In [ ]:
if download_raw_data:
    first_line_index = next(idx for idx, line in enumerate(csv_lines) if line.startswith("Recipient,"))
    first_line_index

In [ ]:
# setting up the data frame, saving locally
if download_raw_data:
    arms_df = pd.read_csv(io.StringIO("\n".join(csv_lines[first_line_index:])))
    arms_df.to_csv("fulldata.csv")
    arms_df
else:
    arms_df = pd.read_json('https://github.com/PUBPOL-2130/notebooks/blob/main/data/sipri_arms_transfers.json?raw=true')
    arms_df.to_csv("fulldata.csv")
    arms_df

In [ ]:
# data cleaning -- mapping transfers to general locations we can use
capitals_map = {
    "ANC (South Africa)*": "South Africa",
    "Anti-Castro rebels (Cuba)*": "Cuba",
    "Amal (Lebanon)*": "Lebanon",
    "Armas (Guatemala)*": "Guatemala",
    "Contras (Nicaragua)*": "Nicaragua",
    "Darfur rebels (Sudan)*": "Sudan",
    "ELF (Ethiopia)*": "Ethiopia",
    "EPLF (Ethiopia)*": "Ethiopia",
    "FRELIMO (Portugal)*": "Portugal",
    "Haiti rebels*": "Haiti",
    "Hezbollah (Lebanon)*": "Lebanon",
    "Houthi rebels (Yemen)*": "Yemen",
    "Indonesia rebels*": "Indonesia",
    "Khmer Rouge (Cambodia)*": "Cambodia",
    "Kurdistan Regional Government (Iraq)*": "Iraq",
    "LF (Lebanon)*": "Lebanon",
    "LRA (Uganda)*": "Uganda",
    "LTTE (Sri Lanka)*": "Sri Lanka",
    "Libya GNC": "Libya",
    "Libya HoR*": "Libya",
    "Congo": "Congo (Brazzaville)",
    "DR Congo": "Congo (Kinshasa)",
    "MNLF (Philippines)*": "Philippines",
    "MPLA (Portugal)*": "Portugal",
    "MTA (Myanmar)*": "Myanmar",
    "Micronesia": "Federated States of Micronesia",
    "Mujahedin (Afghanistan)*": "Afghanistan",
    "NLA (Macedonia)*": "North Macedonia",
    "NTC (Libya)*": "Libya",
    "Northern Alliance (Afghanistan)*": "Afghanistan",
    "Northern Cyprus": "Cyprus",
    "PAIGC (Portugal)*": "Portugal",
    "PIJ (Israel/Palestine)*": "Israel",
    "PKK (Turkiye)*": "Turkey",
    "PLO (Israel)*": "Israel",
    "PRC (Israel/Palestine)*": "Israel",
    "Pathet Lao (Laos)*": "Laos",
    "Provisional IRA (UK)*": "United Kingdom",
    "RPF (Rwanda)*": "Rwanda",
    "RUF (Sierra Leone)*": "United Kingdom",
    "SLA (Lebanon)*": "Lebanon",
    "SNA (Somalia)*": "Somalia",
    "SPLA (Sudan)*": "Sudan",
    "Southern rebels (Yemen)*": "Yemen",
    "Syria rebels*": "Syria",
    "Turkiye": "Turkey",
    "UAE": "United Arab Emirates",
    "UIC (Somalia)*": "Somalia",
    "UNITA (Angola)*": "Angola",
    "Ukraine Rebels*": "Ukraine",
    "United States": "United States of America",
    "United Wa State (Myanmar)*": "Myanmar",
    "Viet Minh (France)*": "France",
    "Viet Nam": "Vietnam",
    "ZAPU (Zimbabwe)*": "Zimbabwe",
    "GUNT (Chad)*": "Chad",
    "FAN (Chad)*": "Chad",
    "FMLN (El Salvador)*": "El Salvador",
    "Gambia": "The Gambia",
    "Lebanon Palestinian rebels*": "Lebanon",
    "Cote d'Ivoire": "Ivory Coast",
    "Bahamas": "The Bahamas",
    "FNLA (Angola)*": "Angola",
    "Cabo Verde": "Cape Verde",
    "Timor-Leste": "East Timor",
    "Saint Vincent": "Saint Vincent and the Grenadines",
    "Guinea-Bissau": "Guinea Bissau",
    "South Vietnam": "Vietnam",  # Saigon is now Ho Chi Minh City
    "Viet Cong (South Vietnam)*": "Vietnam",
    "Hamas (Palestine)*": "Palestine",
    "Soviet Union": "Russia",
    "NATO**": "Belgium",  # NATO headquarters in Brussels
    'European Union**': "Belgium",  # EU headquarters in Brussels
    "OSCE**": "Austria",  # secretariat in Vienna
    "Yemen Arab Republic (North Yemen)": "Yemen",  # same capital as Yemen (Sanaa)
    "North Yemen": "Yemen",  # same capital as Yemen (Sanaa)
    "Czechoslovakia": "Czechia",  # same capital as the modern Czech Republic (Prague)
    "Yugoslavia": "Serbia",  # same capital as Serbia (Belgrade)
    "East Germany (GDR)": "Germany",  # for large-scale flow maps, approximate East Berlin with Berlin
    "Western Sahara": "Morocco",  # largely under Moroccan occupation,
}

exclude_flows = {
    "nan",
    "unknown rebel group*",
    "unknown recipient(s)",
    'unknown supplier(s)',
    "United Nations**",
    "Regional Security System**",
    "African Union**",
    '0.25',
    '3',
}

# (long, lat) coordinates for capitals of entities not included in the places shapefile.
# Several of these entities are countries that no longer exist.
extra_capitals = {
    "Biafra": ("Enugu", 7.5139, 6.4483),  # 1967 capital (now part of Nigeria)
    "Bosnia-Herzegovina": ("Sarajevo", 18.4131, 43.8563),
    "South Yemen": ("Aden", 45.0176, 12.7906),
    "Katanga": ("Lubumbashi", 27.5026, -11.6876),
    "South Sudan": ("Juba",  31.5825, 4.8539),
    "Palestine": ("East Jerusalem", 35.217018, 31.771959),
    "Aruba": ("Oranjestad", -70.0353, 12.5227),
}

In [ ]:
# putting into geodataframe format
extra_capitals_gdf = gpd.GeoDataFrame(
    [
        {
            "adm0name": entity,
            "name": capital,
            "longitude": long,
            "latitude": lat,
            "geometry": Point(long, lat),
        }
        for entity, (capital, long, lat) in extra_capitals.items()
    ],
    crs="epsg:4326",
).set_index("adm0name")

In [ ]:
# reading in simple shapefiles for visualizations
places_gdf = gpd.read_file("https://naciscdn.org/naturalearth/110m/cultural/ne_110m_populated_places_simple.zip")
capitals_gdf = places_gdf[places_gdf["adm0cap"] == 1].set_index("adm0name")
# force each nation to have exactly one capital
capitals_gdf = capitals_gdf[~capitals_gdf["name"].isin(["Sucre", "Yamoussoukro", "Bloemfontein", "Pretoria"])][["name", "latitude", "longitude", "geometry"]]
capitals_gdf = gpd.GeoDataFrame(pd.concat([capitals_gdf, extra_capitals_gdf]), crs="epsg:4326")

In [ ]:
flowmap_arms_df = arms_df[~arms_df["Supplier"].isin(exclude_flows) & ~arms_df["Recipient"].isin(exclude_flows)].rename(
    columns={
        "Year of order": "order_year",
        "Recipient": "recipient",
        "Supplier": "supplier",
        "SIPRI TIV for total order": "order_sipri_tiv"
    }
)
flowmap_arms_df["order_year"] = flowmap_arms_df["order_year"].astype(int)
flowmap_arms_df = flowmap_arms_df[flowmap_arms_df["order_year"] >= 1950]

In [ ]:
orders_by_year_df = flowmap_arms_df.groupby(["order_year", "recipient", "supplier"]).sum()["order_sipri_tiv"]
orders_by_year_df

order_year  recipient  supplier      
1950        Argentina  United States     135.51
            Australia  United Kingdom    337.68
                       United States       0.60
            Belgium    United Kingdom     78.00
                       United States     756.12
                                          ...  
2023        Uruguay    Spain               0.00
                       United States      12.72
            Viet Nam   India              30.00
            Zambia     United States      20.80
            Zimbabwe   China              20.40
Name: order_sipri_tiv, Length: 15215, dtype: float64

## Integrating with FlowmapBlue

Here, we load FlowmapBlue to create beautiful and interactive flowmaps. The steps are broadly similar to what you saw in Week 5. However, now we'll explore different ways that you can filter the data.

In [ ]:
!pip install "git+https://github.com/PUBPOL-2130/notebooks#egg=pubpol2130&subdirectory=lib"

  Cloning https://github.com/PUBPOL-2130/notebooks to /tmp/pip-install-g2929aac/pubpol2130_959286f122de46a098450499be6deca7
  Running command git clone --filter=blob:none --quiet https://github.com/PUBPOL-2130/notebooks /tmp/pip-install-g2929aac/pubpol2130_959286f122de46a098450499be6deca7
  Resolved https://github.com/PUBPOL-2130/notebooks to commit 6cf590f1a0fca797352f11813aa55c035bc3df12
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for pubpol2130: filename=pubpol2130-0.0.1-py3-none-any.whl size=3747 sha256=d7b8e2a2c23e183ad2a63a7eae7b551feede1c8b92f40a665bd35443f1bacc79
  Stored in directory: /tmp/pip-ephem-wheel-cache-ucdadc93/wheels/de/36/19/875166503c3cbba368f467c40c5c908e90c21dca2f0b11f71e
Successfully built pubpol2130
  Attempting uni

As you saw in the Week 5 lab, this line will pop up a dialog asking for permission to generate Google Sheets credentials using your Google login. **You should do this step in Google Colab.**

In [ ]:
from pubpol2130 import google_sheets_credentials, generate_flow_sheet

In [ ]:
flowmap_locations_df = pd.DataFrame(
    [
        {
            "id": loc,
            "name": loc,
            "lat": capitals_gdf.loc[capitals_map.get(loc, loc), "latitude"],
            "lon": capitals_gdf.loc[capitals_map.get(loc, loc), "longitude"],
        }
        for loc in set(flowmap_arms_df["supplier"]) | set(flowmap_arms_df["recipient"])
    ]
)
# we can visualize the first five rows of our location data
flowmap_locations_df.head(5)

,id,name,lat,lon
0,Netherlands,Netherlands,52.351915,4.914694
1,Congo,Congo,-4.257240,15.282744
2,eSwatini,eSwatini,-26.316651,31.133335
3,Haiti,Haiti,18.542971,-72.337980
4,Maldives,Maldives,4.166708,73.499948


In [ ]:
sheet_creds = google_sheets_credentials()

**Now, we can explore filtering the data down so that it's easier to upload to Google sheets.** If you were encountering timeout errors before, this step may be particularly helpful.

Part of what makes filtering this dataframe challenging is that it has what's known as a [`MultiIndex`](https://pandas.pydata.org/docs/user_guide/advanced.html). If we display the index values of the dataframe, you can see that there are three values that identify a row: a year, a recipient, and a supplier.

To avoid indexing and slicing errors, we will use [`loc`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html). However, some of these filtering steps can be down simply with `[]`.


In [ ]:
orders_by_year_df.index

MultiIndex([(1950,      'Argentina',  'United States'),
            (1950,      'Australia', 'United Kingdom'),
            (1950,      'Australia',  'United States'),
            (1950,        'Belgium', 'United Kingdom'),
            (1950,        'Belgium',  'United States'),
            (1950,         'Brazil',  'United States'),
            (1950,       'Bulgaria',   'Soviet Union'),
            (1950,         'Canada', 'United Kingdom'),
            (1950,         'Canada',  'United States'),
            (1950,          'Chile',         'Canada'),
            ...
            (2023, 'United Kingdom',  'United States'),
            (2023,  'United States',         'Canada'),
            (2023,  'United States',         'Israel'),
            (2023,  'United States',          'Japan'),
            (2023,        'Uruguay',        'Denmark'),
            (2023,        'Uruguay',          'Spain'),
            (2023,        'Uruguay',  'United States'),
            (2023,       'Viet N

You can view the different values for each "level" of the index with the function `get_level_values()`. Remember, Python indexing starts with 0!

In [ ]:
orders_by_year_df.index.get_level_values(0)

Index([1950, 1950, 1950, 1950, 1950, 1950, 1950, 1950, 1950, 1950,
       ...
       2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023],
      dtype='int64', name='order_year', length=15215)

In [ ]:
orders_by_year_df.index.get_level_values(1)

Index(['Argentina', 'Australia', 'Australia', 'Belgium', 'Belgium', 'Brazil',
       'Bulgaria', 'Canada', 'Canada', 'Chile',
       ...
       'United Kingdom', 'United States', 'United States', 'United States',
       'Uruguay', 'Uruguay', 'Uruguay', 'Viet Nam', 'Zambia', 'Zimbabwe'],
      dtype='object', name='recipient', length=15215)

In [ ]:
orders_by_year_df.index.get_level_values(2)

Index(['United States', 'United Kingdom', 'United States', 'United Kingdom',
       'United States', 'United States', 'Soviet Union', 'United Kingdom',
       'United States', 'Canada',
       ...
       'United States', 'Canada', 'Israel', 'Japan', 'Denmark', 'Spain',
       'United States', 'India', 'United States', 'China'],
      dtype='object', name='supplier', length=15215)

You can also filter different values of the index to get subsets of the data. For example, the following code subsets the data to orders that occurred between 1950 and 1970.

Note: using `range()` in Python is an easy way to create a sequence of values. To view them in a list format, we can call `list()` as well.

In [ ]:
orders_by_year_df.loc[range(1950, 1970)]

order_year  recipient   supplier      
1950        Argentina   United States      135.51
            Australia   United Kingdom     337.68
                        United States        0.60
            Belgium     United Kingdom      78.00
                        United States      756.12
                                           ...   
1969        Viet Nam    Czechoslovakia      12.00
                        Soviet Union      1659.50
            Yugoslavia  Soviet Union         9.50
            Zambia      Italy               22.62
                        United Kingdom       8.00
Name: order_sipri_tiv, Length: 2617, dtype: float64

In [ ]:
# range will produce a sequence of values from 1950 to 1970
range(1950, 1970)

range(1950, 1970)

In [ ]:
# calling list will put them into a list format
list(range(1950, 1970))

[1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969]

Here, we subset to all orders where the United States was the recipient.

In [ ]:
orders_by_year_df.loc[:,'United States']

order_year  supplier                
1950        United Kingdom              2760.00
1951        Canada                       281.25
            United Kingdom              3830.00
1952        France                       500.00
1953        Canada                        19.00
1954        United Kingdom               180.00
1955        Canada                        40.00
1958        France                       500.00
1959        Canada                        15.00
            France                        50.00
1960        Canada                       596.25
1961        France                       150.00
1962        Switzerland                   11.70
1965        Canada                        52.88
            United Kingdom              2680.00
1966        United Kingdom                90.00
1967        Canada                        33.50
            Taiwan                        12.00
1969        Switzerland                    6.40
            United Kingdom              1239.30
1970        Canada                       110.00
            France                        18.00
1971        Switzerland                    4.00
1973        Italy                        205.00
            Netherlands                  510.00
            United Kingdom               275.40
1975        Germany                      341.40
            United Kingdom               900.00
1976        Egypt                        102.72
            France                       430.50
1978        Canada                         0.70
            Egypt                         20.80
            Switzerland                    4.10
1979        France                       328.25
1980        United Kingdom               184.34
1981        Canada                         0.54
            France                       144.00
            United Kingdom              2410.34
1982        Canada                       115.29
1983        United Kingdom                34.34
1984        Canada                        88.02
            France                       160.00
            Israel                        57.60
            Italy                        165.00
            Netherlands                    5.00
            United Kingdom               212.70
1985        Australia                    124.60
            Canada                        18.54
            United Kingdom              4341.34
1986        Israel                        67.20
            Italy                        651.12
            Poland                         5.44
            Spain                         41.80
1987        Canada                         3.24
            Chad                          19.56
            Switzerland                   80.00
            United Kingdom               452.72
1988        Germany                        1.75
            Israel                        77.40
            United Kingdom               102.00
1989        France                        42.75
            Norway                        32.00
            United Kingdom               110.40
1990        Canada                        12.50
            Germany                        8.40
            Israel                       313.50
            Italy                        129.68
            Japan                        648.00
            Norway                        33.00
            United Kingdom               172.00
1991        Australia                     42.00
            Germany                      176.55
            United Kingdom               107.50
1992        Israel                        18.60
            Mujahedin (Afghanistan)*       0.80
            United Kingdom                68.08
1993        Canada                        71.40
            Denmark                      429.20
            Germany                       24.05
            Israel                        45.00
            Jordan                        35.60
            United Kingdom                77.20
1994        Canada         

And here we can filter to all cases where the United States is the supplier.

In [ ]:
orders_by_year_df.loc[:,:,'United States']

order_year  recipient     
1950        Argentina          135.51
            Australia            0.60
            Belgium            756.12
            Brazil             247.20
            Canada            2085.60
                               ...   
2023        UAE                 22.39
            Ukraine            250.82
            United Kingdom     100.00
            Uruguay             12.72
            Zambia              20.80
Name: order_sipri_tiv, Length: 3437, dtype: float64

Now you can try slicing and filtering your data in order to avoid potential timeout issues when creating a Google sheet!

As an example, let's try filtering for:
* recent arms orders that occurred in the last 5 years (the dataset doesn't include information from 2024 or 2025)
* involving the United States as a supplier

In [ ]:
orders_by_year_df_filtered = orders_by_year_df.loc[range(2019,2024),:,['United States']]
orders_by_year_df_filtered

order_year  recipient           supplier     
2019        Afghanistan         United States      33.15
            Argentina           United States       3.30
            Australia           United States      64.80
            Bahrain             United States     744.46
            Belgium             United States       1.00
            Bosnia-Herzegovina  United States      12.40
            Brazil              United States       0.24
            Bulgaria            United States     336.62
            Canada              United States     293.50
            Chad                United States       1.00
            Chile               United States      22.40
            Czechia             United States     110.00
            Denmark             United States      16.80
            Egypt               United States      37.00
            El Salvador         United States       4.08
            Estonia             United States       6.09
            Finland             United States      80.00
            France              United States       2.50
            Germany             United States     657.00
            India               United States      54.00
            Indonesia           United States     204.15
            Iraq                United States      11.00
            Israel              United States      80.00
            Italy               United States       7.80
            Jamaica             United States      10.80
            Japan               United States    1018.05
            Jordan              United States      33.60
            Kenya               United States       1.44
            Kuwait              United States      36.00
            Latvia              United States      29.00
            Libya GNC           United States       0.75
            Lithuania           United States      36.00
            Luxembourg          United States       0.28
            Malaysia            United States       0.48
            Mali                United States       5.00
            Montenegro          United States      12.06
            Morocco             United States      25.00
            Netherlands         United States     526.60
            Niger               United States      10.20
            Nigeria             United States       4.00
            Norway              United States      36.00
            Oman                United States       0.21
            Pakistan            United States       2.00
            Peru                United States       1.98
            Philippines         United States     117.32
            Poland              United States     257.36
            Qatar               United States     222.92
            Romania             United States      11.20
            Saudi Arabia        United States     410.00
            South Korea         United States     324.00
            Spain               United States     179.57
            Taiwan              United States    3211.12
            Thailand            United States      52.99
            Tunisia             United States       0.45
            Turkmenistan        United States      18.00
            UAE                 United States     843.20
            Uganda              United States       1.50
            Ukraine             United States      15.54
            United Kingdom      United States     756.60
            Uruguay             United States       1.48
            Viet Nam            United States      54.24
2020        Albania             United States       7.80
            Argentina           United States      13.68
            Australia           United States     186.00
            Bangladesh          United States       2.20
            Belgium             United States     102.46
            Brazil              United States      12.16
            Brunei              United States       0.35
            Canada              United States      39.00
            Chad     

Now, we can upload our **filtered** data to Google sheets. This process should be much faster!

In [ ]:
# upload our filtered data to Google sheets
flow_sheet = generate_flow_sheet(
    sheet_creds=sheet_creds,
    locations_df=flowmap_locations_df,
    created_by_name="",  # YOUR NAME HERE
    created_by_email="", # YOUR EMAIL HERE
    data_source_name="SIPRI Arms Transfers Database",
    data_source_url="https://www.sipri.org/databases/armstransfers",
    incoming_tooltip="Inbound arms transfers (TIV)",
    outgoing_tooltip="Outbound arms transfers (TIV)",
    flow_tooltip="Arms transfer (TIV)",
    total_unit="TIVs",
    sheet_title="PUBPOL 2130: SIPRI arms transfers (orders over time)",
    flow_title="SIPRI Arms Transfers Database: orders over time",
    flows={
        f"Year: {year}": year_df.reset_index().rename(columns={
            "supplier": "origin",
            "recipient": "dest",
            "order_sipri_tiv": "count",
        })
        # note that we replaced the dataframe here
        for year, year_df in orders_by_year_df_filtered.groupby(level=0)
    }
)

APIError: {'code': 403, 'message': 'The user does not have sufficient permissions for this file.', 'errors': [{'message': 'The user does not have sufficient permissions for this file.', 'domain': 'global', 'reason': 'insufficientFilePermissions'}]}

In [ ]:
print(flow_sheet.url)

In [ ]:
print(f"https://www.flowmap.blue/{flow_sheet.url.split('/')[-1]}")